In [2]:
import pandas as pd

/var/folders/q5/ndxjp9156hqctcf2kg6b6lvw0000gn/T/ipykernel_65429/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Perform exploratory statistical analysis on NFL betting odds data
All hypotheses apply to the date range 2012 to 2021.

In [39]:
csv_data = pd.read_csv('nfl_odds_data.csv')

# drop rows where game was a spread tie
csv_data = csv_data[csv_data['spread_tie'] != True]
csv_data = csv_data[csv_data['spread'] != 'PK']

In [43]:
def best_spreads(csv_data):
    # Get unique spread values
    unique_spreads = csv_data['spread'].unique()

    # Initialize a list to store spread, odds, and occurrences tuples
    spread_odds_occurrences_list = []

    # Iterate over sorted spread values
    for specific_spread in unique_spreads:
        # Filter rows based on the specific spread
        filtered_rows = csv_data[csv_data['spread'] == specific_spread]
        
        # Calculate the odds that 'underdog' equals 'teamCovered'
        total_rows = len(filtered_rows)
        matching_underdog_rows = len(filtered_rows[filtered_rows['underdog'] == filtered_rows['team_covered']])
        
        if total_rows > 0:
            odds_underdog = matching_underdog_rows / total_rows
        else:
            odds_underdog = 0

        # Calculate the occurrences for 'underdog' equals 'teamCovered'
        occurrences_underdog = matching_underdog_rows

        # Calculate the odds that 'favorite' equals 'teamCovered'
        matching_favorite_rows = len(filtered_rows[filtered_rows['favorite'] == filtered_rows['team_covered']])
        
        if total_rows > 0:
            odds_favorite = matching_favorite_rows / total_rows
        else:
            odds_favorite = 0

        # Calculate the occurrences for 'favorite' equals 'teamCovered'
        occurrences_favorite = matching_favorite_rows

        # Append spread, odds, and occurrences tuples to the list
        if occurrences_favorite and occurrences_underdog > 4:
            spread_odds_occurrences_list.append((specific_spread, odds_underdog, occurrences_underdog, odds_favorite, occurrences_favorite))

    # Print the results
    for spread, odds_underdog, occurrences_underdog, odds_favorite, occurrences_favorite in spread_odds_occurrences_list:
        print(f"Spread: {spread}, Odds Underdog: {odds_underdog}, Occurrences Underdog: {occurrences_underdog}, Odds Favorite: {odds_favorite}, Occurrences Favorite: {occurrences_favorite}")

best_spreads(csv_data)

Spread: -8.5, Odds Underdog: 0.5185185185185185, Occurrences Underdog: 14, Odds Favorite: 0.48148148148148145, Occurrences Favorite: 13
Spread: -3.5, Odds Underdog: 0.5541666666666667, Occurrences Underdog: 133, Odds Favorite: 0.44583333333333336, Occurrences Favorite: 107
Spread: -6.5, Odds Underdog: 0.4852941176470588, Occurrences Underdog: 66, Odds Favorite: 0.5147058823529411, Occurrences Favorite: 70
Spread: -4.0, Odds Underdog: 0.5555555555555556, Occurrences Underdog: 60, Odds Favorite: 0.4444444444444444, Occurrences Favorite: 48
Spread: -9.0, Odds Underdog: 0.6363636363636364, Occurrences Underdog: 21, Odds Favorite: 0.36363636363636365, Occurrences Favorite: 12
Spread: -3.0, Odds Underdog: 0.49696969696969695, Occurrences Underdog: 164, Odds Favorite: 0.503030303030303, Occurrences Favorite: 166
Spread: -1.5, Odds Underdog: 0.4945054945054945, Occurrences Underdog: 45, Odds Favorite: 0.5054945054945055, Occurrences Favorite: 46
Spread: -5.5, Odds Underdog: 0.4854368932038835,

In [4]:
def team_stats_analysis(csv_data):
    # create DataFrame with unique teams
    teams = pd.concat([csv_data['favorite'], csv_data['underdog'], csv_data['team_covered']]).unique()
    team_stats = pd.DataFrame({'Team': teams, 'Underdog Count': 0, 'Covered Spread Count': 0})
    
    # underdog count for each underdog team
    team_stats['Underdog Count'] = csv_data.groupby('underdog').size().reindex(teams, fill_value=0).values
    
    # covered spread count for each team that covered
    team_stats['Covered Spread Count'] = csv_data.groupby('team_covered').size().reindex(teams, fill_value=0).values
    
    return team_stats
print(team_stats_analysis(csv_data))

                        Team  Underdog Count  Covered Spread Count
0       Tampa Bay Buccaneers              87                    76
1            Atlanta Falcons              72                    70
2              Buffalo Bills              83                    82
3          Carolina Panthers              77                    76
4        San Francisco 49ers              80                    74
5       Jacksonville Jaguars             119                    63
6           Seattle Seahawks              42                    85
7           Tennessee Titans              82                    65
8   Washington Football Team              25                    15
9         Kansas City Chiefs              41                    80
10      New England Patriots              24                    89
11         Green Bay Packers              43                    86
12            Denver Broncos              67                    82
13          Los Angeles Rams              23                  

In [6]:
def underdog_coverage_percentage_per_season(csv_data):
    # DataFrame with the underdog and whether it covered the spread for each game
    underdog_coverage_df = csv_data[['season', 'underdog', 'team_covered']].copy()

    # binary column indicating whether underdog covered spread
    underdog_coverage_df['Underdog Covered'] = (underdog_coverage_df['team_covered'] == underdog_coverage_df['underdog'])

    # underdog coverage percentage for each season
    result = underdog_coverage_df.groupby('season')['Underdog Covered'].mean() * 100

    return result
print(underdog_coverage_percentage_per_season(csv_data))

season
2012    51.428571
2013    47.325103
2014    50.406504
2015    51.428571
2016    48.163265
2017    44.855967
2018    52.479339
2019    53.688525
2020    54.400000
2021    50.191571
Name: Underdog Covered, dtype: float64


##### Hypothesis
Betting only the underdog on every game, every season, would have yielded net positive returns
##### Conclusion
False. Would have been up 0.44%, but after betting fees (typically at least 3%), would have lost money

In [5]:
def underdog_coverage_percentage(csv_data):
    total_games = len(csv_data)
    underdog_covered_games = len(csv_data[csv_data['team_covered'] == csv_data['underdog']])

    # percentage of games where underdog covers
    underdog_coverage_percent = (underdog_covered_games / total_games) * 100
    return underdog_coverage_percent
print(underdog_coverage_percentage(csv_data))

50.44642857142857


Any team has never been on a > 13 cover or failure to cover streak, so having observed a streak of 13, bet on the next outcome for that team to be opposite of the last outcome

In [7]:
def max_win_loss_streaks(csv_data):
    # find every unique team who covered
    unique_teams = csv_data.team_covered.unique()
    
    # will contain combined team DataFrames at the end
    final_df = pd.DataFrame()

    for team in unique_teams:
        
        # get all rows where team played a game
        where_played = csv_data[(csv_data['straight_winner'] == team) | (csv_data['straight_loser'] == team)]
        
        # Check 'straight_winner' | 'straight_loser' == team & 'team_covered' == team
        condition = ((where_played['straight_winner'] == team) | (where_played['straight_loser'] == team)) & (where_played['team_covered'] == team)
        false_condition = ~condition
        
        # find largest streak of consecutive true
        win_streak_lengths = condition.groupby(condition.cumsum()).cumcount() + 1
        win_max_streak = win_streak_lengths.max()
        
        # find largest streak of consecutive false
        loss_streak_lengths = false_condition.groupby(false_condition.cumsum()).cumcount() + 1
        loss_max_streak = loss_streak_lengths.max()
        
        # create temp DataFrame to store the results
        result_df = pd.DataFrame({
            'Team': [team],
            'Max_Win_Streak': [win_max_streak],
            'Max_Loss_Streak': [loss_max_streak]
        })
        
        # concatenate result_df with final_df
        final_df = pd.concat([final_df, result_df], ignore_index=True)
    
    return final_df
        

print(max_win_loss_streaks(csv_data))

                        Team  Max_Win_Streak  Max_Loss_Streak
0             Dallas Cowboys               9               11
1        Philadelphia Eagles               8                8
2        Pittsburgh Steelers               6                6
3          Carolina Panthers              10                8
4              Detroit Lions               9                6
5             Houston Texans               8                6
6           Seattle Seahawks               5               11
7          Arizona Cardinals               7                9
8       Los Angeles Chargers               7                5
9           Cleveland Browns               9                5
10            Miami Dolphins               7                6
11        New Orleans Saints               6               10
12            Denver Broncos               9                6
13          Los Angeles Rams               8                5
14         Las Vegas Raiders               5                5
15      

In [60]:
def who_covers(csv_data, team_type, cover_type):
    # count total number of games and number of covers based on team_type and cover_type
    total_games = len(csv_data)
    covers = csv_data[
        (csv_data[cover_type] == csv_data['at']) & (csv_data['team_covered'] == csv_data[cover_type])
    ].shape[0]

    # handle division by zero
    if total_games == 0:
        return 0.0

    # calculate percentage
    percentage = (covers / total_games) * 100
    return percentage * 2

print(who_covers(csv_data, 'home', 'underdog'))
print(who_covers(csv_data, 'home', 'favorite'))


35.198040016333195
61.331155573703555


If the underdog or favorite has the same outcome 10 times in a row, bet on the next outcome to be opposite of the last outcome

In [12]:
def under_favorite_win_loss_streaks(csv_data):
    
        # check 'straight_winner' | 'straight_loser' == team & 'team_covered' == team
        condition = (csv_data['underdog'] == csv_data['team_covered'])
        false_condition = ~condition
        
        # find largest streak of consecutive true
        underdog_covered_streak_lengths = condition.groupby(condition.cumsum()).cumcount() + 1
        underdog_covered_max_streak = underdog_covered_streak_lengths.max()
        
        # find largest streak of consecutive false
        favorite_covered_streak_lengths = false_condition.groupby(false_condition.cumsum()).cumcount() + 1
        favorite_covered_max_streak = favorite_covered_streak_lengths.max()
    
        print(f"Max underdog cover streak: {underdog_covered_max_streak}\nMax favorite cover streak: {favorite_covered_max_streak}")
        
under_favorite_win_loss_streaks(csv_data)

Max underdog cover streak: 10
Max favorite cover streak: 9
